List of kart numbers and its average best lap time, from the last 30 days

In [11]:
import sqlite3
import pandas as pd
from datetime import timedelta

In [2]:
conn = sqlite3.connect('../scraping/scraping_data.db')
cursor = conn.cursor()

In [30]:
df = pd.read_sql_query("select * from params_to_scrap where fetched is True", conn)
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
end_date = df['date'].max()
init_date = end_date - timedelta(days=30)
print(end_date, init_date)

2023-12-31 00:00:00 2023-12-01 00:00:00


In [50]:
query = f"""
with last_30_days as (
    select *
    from
        (select *,
        date(year || '-' || substr('00' || month, -2) || '-' || substr('00' || day, -2)) as date
        from params_to_scrap
        where 
        fetched is True) t
    where date >= '{init_date.strftime('%Y-%m-%d')}'
)
select 
btr.car_number,
btr.best_lap_time
from last_30_days l
left join bronze_racing_tries brt on brt.params_id = l.id
left join bronze_try_results btr on btr.uid = brt.uid
"""
df = pd.read_sql_query(query, conn)
df

,car_number,best_lap_time
0,000,1:00.457
1,001,58.780
2,008,1:02.119
3,012,59.808
4,017,59.322
...,...,...
8420,059,1:03.382
8421,062,1:05.720
8422,063,1:15.777
8423,064,1:21.705
